# Algorytmy ewolucyjne i metaheurystyczne - Heurystyki konstrukcyjne

**Install libs**

In [1]:
!pip install requests
!pip install pandas
!pip install matplotlib
!pip install numpy
!pip install scipy
!pip install sklearn
!pip install pandoc

**Get data url:**

In [2]:
import requests

target_url_kroa100 = "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/kroA100.tsp"
target_url_krob100 = "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/kroB100.tsp"

def get_file_from_url(target_url):
    file = requests.get(target_url)
    return file.text
    

**Output test:**

In [3]:
data = get_file_from_url(target_url_krob100)
for line in data.split("\n")[6:-2]:
    print(line.split())

['1', '3140', '1401']
['2', '556', '1056']
['3', '3675', '1522']
['4', '1182', '1853']
['5', '3595', '111']
['6', '962', '1895']
['7', '2030', '1186']
['8', '3507', '1851']
['9', '2642', '1269']
['10', '3438', '901']
['11', '3858', '1472']
['12', '2937', '1568']
['13', '376', '1018']
['14', '839', '1355']
['15', '706', '1925']
['16', '749', '920']
['17', '298', '615']
['18', '694', '552']
['19', '387', '190']
['20', '2801', '695']
['21', '3133', '1143']
['22', '1517', '266']
['23', '1538', '224']
['24', '844', '520']
['25', '2639', '1239']
['26', '3123', '217']
['27', '2489', '1520']
['28', '3834', '1827']
['29', '3417', '1808']
['30', '2938', '543']
['31', '71', '1323']
['32', '3245', '1828']
['33', '731', '1741']
['34', '2312', '1270']
['35', '2426', '1851']
['36', '380', '478']
['37', '2310', '635']
['38', '2830', '775']
['39', '3829', '513']
['40', '3684', '445']
['41', '171', '514']
['42', '627', '1261']
['43', '1490', '1123']
['44', '61', '81']
['45', '422', '542']
['46', '2698',

**Get lat lon from data**

In [4]:
def get_lat_lon_from_url(target_url):
    data = get_file_from_url(target_url)
    list_of_cords = data.split("\n")[6:-2]
    return zip(*[(int(i.split()[1]), int(i.split()[2])) for i in list_of_cords])


**Calculate matrix**

In [5]:
import pandas as pd
import numpy as np
import math

def calc_matrix(lat, lon):
    matrix_size = len(lat)
    matrix = pd.DataFrame(np.nan,index=np.arange(matrix_size), columns=np.arange(matrix_size))
    for row in range(matrix.shape[0]):
        for col in range(row, matrix.shape[1]):
            if row == col:
                matrix.iat[row, col] = np.inf
            else:
                matrix.iat[row, col] = matrix.iat[col, row] =  round(math.sqrt(((lat[col]-lat[row])**2)+((lon[col]-lon[row])**2)))
    
    return matrix

**Display steps dynamically**

In [6]:
import matplotlib.pyplot as plt

def test_display_dynamically(tedis,distance_matrix, lat, lon):
    from IPython.display import clear_output
    for elem in tedis:
        clear_output(wait=True)
        x,y=[],[]
        plt.scatter(lat, lon, color="black")
        for i,vertex in enumerate(elem):
            x.append(lat[vertex])
            y.append(lon[vertex])
            plt.scatter(lat[vertex], lon[vertex],color='red',zorder=2)
            plt.annotate(i, (lat[vertex], lon[vertex]))
        plt.scatter(lat[elem[0]], lon[elem[0]],color='green',zorder=2)
        plt.plot(x,y,zorder=1)
        
        path_length = calculate_path_length(elem, distance_matrix)
        plt.figtext(1.02, 0.5, "Path length {path_length}".format(path_length=path_length),horizontalalignment ="center", 
            wrap = True, fontsize = 10,  
            bbox ={'facecolor':'grey',  
                   'alpha':0.3, 'pad':5})
        plt.pause(0.02)
    plt.show()
    return True

In [7]:
def get_combs(current_tsp):
    return [[current_tsp[i],current_tsp[i+1]] for i,d in enumerate(current_tsp[:-1])]

def calculate_path_length(tsp_final_outcome, distance_matrix):
    return sum([distance_matrix.loc[x,y].sum() for (x,y) in get_combs(tsp_final_outcome)])

**NN Algorithm:**

In [8]:
def get_clossest_points(distance_matrix, idx):
    num = round(0.5*distance_matrix.shape[0] - 1)
    neigbrs = distance_matrix.nsmallest(num, idx)
    neigbrs_list = list(neigbrs.index)
    return neigbrs_list

In [9]:
import random

def tsp_nn(distance_matrix, selected_vertex = None):
    if selected_vertex is None:
        selected_vertex = random.randint(0,99)

    #get rid off the furthest cords
    closest_points = get_clossest_points(distance_matrix, selected_vertex)
    distance_matrix_truncated = distance_matrix.loc[closest_points+[selected_vertex], closest_points+[selected_vertex]]
    
    tsp_outcome = [selected_vertex]
    test_display = [tsp_outcome.copy()]
    for _ in range(distance_matrix_truncated.shape[0]-1):
        #find minimum in column
        ix_min = int(distance_matrix_truncated[[selected_vertex]].idxmin()) 
        #push min to outcome
        tsp_outcome.append(ix_min)
        test_display.append(tsp_outcome.copy())
        #set distance to 0
        distance_matrix_truncated = distance_matrix_truncated.drop(index=selected_vertex, columns=selected_vertex)
        #set vertex as ix_min
        selected_vertex = ix_min

    tsp_outcome.append(tsp_outcome[0])
    test_display[-1].append(test_display[-1][0])
    return tsp_outcome, test_display

**NN ALG Run:**

%matplotlib inline

import matplotlib.pyplot as plt
from IPython.display import display, HTML

distance_matrix = calc_matrix(lat,lon)


for i in range(10):
    tsp, ted = tsp_nn(distance_matrix,selected_vertex = i)
    print(len(tsp))

#test_display_dynamically(ted, distance_matrix,nn=True)

**Greedy Cycle Alg**

In [10]:
import pandas as pd
import numpy as np

def get_index_min_value_in_cols(list_of_cols, distance_matrix):
    distance_matrix_truncated = distance_matrix.drop(list(set(list_of_cols)))
    min_value_list = distance_matrix_truncated[list_of_cols].min()
    min_value = min_value_list.min()
    min_value_index_in_list = min_value_list.to_list().index(min_value)
    col_index_in_distance_matrix = list_of_cols[min_value_index_in_list]
    ix_min = int(distance_matrix_truncated[col_index_in_distance_matrix].idxmin()) 
    return ix_min

In [11]:
def get_min_length_to_point(point,current_tsp, distance_matrix):
    minimal_points = []
    current_minimal = 1000000000
    for (x,y) in get_combs(current_tsp):
        distance_matrix_truncated = distance_matrix.loc[[point], [x,y]]
        new_length = distance_matrix_truncated.sum().sum()
        x_y_distance = distance_matrix.loc[x,y].sum()
        total_length_increase =  new_length - x_y_distance
        if total_length_increase < current_minimal:
            minimal_points = [x,y]
            current_minimal = total_length_increase
    return minimal_points

def insert_after_element(current_tsp, elem, value_to_insert):
    elem_index = current_tsp.index(elem)
    current_tsp.insert(elem_index + 1, value_to_insert)
    return current_tsp

In [12]:
import random
import numpy as np

def tsp_greedy(distance_matrix,selected_vertex = None):
    if selected_vertex is None:
        selected_vertex = random.randint(0,99)
    
    #get rid off the furthest cords
    closest_points = get_clossest_points(distance_matrix, selected_vertex)
    distance_matrix_truncated = distance_matrix.loc[closest_points+[selected_vertex], closest_points+[selected_vertex]]

    #find min distannce and connect
    ix_start_min = int(distance_matrix_truncated[[selected_vertex]].idxmin()) 
    tsp_outcome = [selected_vertex, ix_start_min, selected_vertex]

    test_display = [tsp_outcome.copy()]
    for _ in range(distance_matrix_truncated.shape[0]-2):
        
        #find minimum in current tsp outcome
        ix_min = int(get_index_min_value_in_cols(tsp_outcome, distance_matrix_truncated)) 
 
        #find suitable place to insert elem
        x,y = get_min_length_to_point(ix_min,tsp_outcome,distance_matrix_truncated)
        
        #push min to outcome
        tsp_outcome = insert_after_element(tsp_outcome,x,ix_min)
        test_display.append(tsp_outcome.copy())

        
    return tsp_outcome, distance_matrix_truncated,test_display

%matplotlib inline

import matplotlib.pyplot as plt
from IPython.display import display, HTML

distance_matrix = calc_matrix(lat,lon)
tsp, d,tedis= tsp_greedy(distance_matrix)
print(len(tsp))
w = list(set(tsp) - set(d.columns))
print("W",w)
t = list(set([i for i in range(0,100)]) - set(tsp))
print("t",len(t))



**Regret ALG**

In [13]:
def get_cost(point,current_tsp, distance_matrix):
    minimal_points = []
    for (x,y) in get_combs(current_tsp):
        distance_matrix_truncated = distance_matrix.loc[[point], [x,y]]
        new_length = distance_matrix_truncated.sum().sum()
        x_y_distance = distance_matrix.loc[x,y].sum()
        total_cost =  new_length - x_y_distance
        minimal_points.append((total_cost,x,y,point))
    #sort
    minimal_points.sort(key = lambda x: x[0]) 
    #2 regret
    diff = minimal_points[1][0] - minimal_points[0][0]
    x = minimal_points[0][1]
    y = minimal_points[0][2]
    point = minimal_points[0][3]
    regret  = (diff, x, y, point)
    return regret

In [14]:
def tsp_regret(distance_matrix,selected_vertex = None):
    if selected_vertex is None:
        selected_vertex = random.randint(0,99)
    
    #get rid off the furthest cords
    closest_points = get_clossest_points(distance_matrix, selected_vertex)
    distance_matrix_truncated = distance_matrix.loc[closest_points+[selected_vertex], closest_points+[selected_vertex]]
    #find min distannce and connect
    ix_start_min = int(distance_matrix_truncated[[selected_vertex]].idxmin()) 
    tsp_outcome = [selected_vertex, ix_start_min, selected_vertex]

    closest_points.remove(ix_start_min)
    test_display = [tsp_outcome.copy()]
    for _ in range(distance_matrix_truncated.shape[0]-2):
        
        regret_scores = []
        for point in closest_points:
            regret_scores.append(get_cost(point, tsp_outcome, distance_matrix))
        regret_scores.sort(key = lambda x: x[0], reverse=True) 
        _,x,_,point = regret_scores[0]
        #push min to outcome
        tsp_outcome = insert_after_element(tsp_outcome,x,point)
        test_display.append(tsp_outcome.copy())
        #delete vertex from closest
        closest_points.remove(point)
        
    return tsp_outcome, distance_matrix_truncated, test_display

%matplotlib inline

import matplotlib.pyplot as plt
from IPython.display import display, HTML

distance_matrix = calc_matrix(lat,lon)
tsp,d, te = tsp_regret(distance_matrix)
print(len(tsp))
w = list(set(tsp) - set(d.columns))
print("W",w)
t = list(set([i for i in range(0,100)]) - set(tsp))
print("t",len(t))
#x,y = get_plot_points(tsp, lat, lon)

test_display_dynamically(te)

test_display_dynamically(te)

"""
TEST RUN
"""
import random

def create_comparison_table(target_url, name):    
    lat, lon = get_lat_lon_from_url(target_url)
    distance_matrix = calc_matrix(lat,lon)
    final_scores = []
    
    
    vertexes = [i for i  in range(int(len(lat)))]
    random.shuffle(vertexes)
    #sample of vertexes
    vertexes = vertexes[:50]
    
    for vertex in vertexes:
        print(type(vertex), vertex)
        tspnn,_  = tsp_nn(distance_matrix,selected_vertex = vertex)
        tspgreedy,_,_  = tsp_greedy(distance_matrix, selected_vertex =  vertex)
        tspregret,_,_  = tsp_regret(distance_matrix, selected_vertex=vertex)
        scores = [calculate_path_length(tspnn, distance_matrix),calculate_path_length(tspgreedy, distance_matrix),calculate_path_length(tspregret, distance_matrix)]
        final_scores.append(scores.copy())
    df = pd.DataFrame(np.array(final_scores),
                       columns=['NN', 'GREEDY', 'REGRET'],
                        index = vertexes)
    display(df)
    df.to_csv(name, index=True) 


create_comparison_table(target_url=target_url_kroa100, name = "kroa")
create_comparison_table(target_url=target_url_krob100, name = "krob")

In [20]:
def create_plot(tsp_outcome, lat, lon, title, distance_matrix):
    plt.cla()
    x,y=[],[]
    plt.scatter(lat, lon, color="black")
    for i,vertex in enumerate(tsp_outcome):
        #print(vertex,lat[vertex],lon[vertex])
        x.append(lat[vertex])
        y.append(lon[vertex])
        plt.scatter(lat[vertex], lon[vertex],color='red',zorder=2)
        plt.annotate(i, (lat[vertex], lon[vertex]))
    x.append(x[0])
    y.append(y[0])
    plt.scatter(lat[tsp_outcome[0]], lon[tsp_outcome[0]],color='green',zorder=2)
    plt.plot(x,y,zorder=1)
    
    path_length = calculate_path_length(tsp_outcome, distance_matrix)
    #plt.figtext(0.0, 1.0, "Path length {path_length}".format(path_length=path_length),horizontalalignment ="center", 
#             wrap = True, fontsize = 10,  
#             bbox ={'facecolor':'grey',  
#                    'alpha':0.3, 'pad':5})
    
    plt.title(f'{title} \n Długość ścieżki: {path_length}')
    plt.savefig(f'{title}.png', format='png', dpi=1200)
       
    
    return x,y

In [ ]:
def test_length_tt():
    lat, lon = get_lat_lon_from_url(target_url_kroa100)
    print(lat[1])
    distance_matrix = calc_matrix(lat,lon)
    tsp,tedis = tsp_nn(distance_matrix, 42)
    print(tsp)
    
    #print(calculate_path_length(tsp[:-1], distance_matrix))
    test_display_dynamically(tedis, distance_matrix, lat, lon)
    #print(calculate_path_length(tsp, distance_matrix))
test_length_tt()

import pandas as pd

def prepare(df, index,title1,target_url):
    l = df['Unnamed: 0'].to_list()
    print(l)
    l.sort()
    print(l)
    lat, lon = get_lat_lon_from_url(target_url)
    distance_matrix = calc_matrix(lat,lon)
    tsp,tedis = tsp_nn(distance_matrix, l[index])
    create_plot(tsp, lat,lon, title1, distance_matrix)
    
c = pd.read_csv("kroa", index_col=False)
display(c)
print(c.describe())
nn = c.loc[c['NN'].isin([11457 ,15548 ])]
prepare(nn, 0,"Wynik dla najkrótszej ścieżki - algorytm najbliższego sąsiada",target_url=target_url_kroa100)
prepare(nn, 1,"Wynik dla najdłuższej ścieżki - algorytm najbliższego sąsiada",target_url=target_url_kroa100)
print()
c = pd.read_csv("kroa", index_col=False)      
nn = c.loc[c['GREEDY'].isin([11838 ,14373])]
prepare(nn, 1,"Wynik dla najkrótszej ścieżki - algorytm rozbudowy cyklu",target_url=target_url_kroa100)
prepare(nn, 0,"Wynik dla najdłuższej ścieżki - algorytm rozbudowy cyklu",target_url=target_url_kroa100)
print(nn)
c = pd.read_csv("kroa", index_col=False)
nn = c.loc[c['REGRET'].isin([10472,12794])]
prepare(nn, 0,"Wynik dla najkrótszej ścieżki - algorytm oparty na żalu",target_url=target_url_kroa100)
prepare(nn, 1,"Wynik dla najdłuższej ścieżki - algorytm oparty na żalu",target_url=target_url_kroa100)
print(nn)


c = pd.read_csv("krob", index_col=False)

nn = c.loc[c['NN'].isin([10926 ,17955 ])]
prepare(nn, 1,"Wynik dla najkrótszej ścieżki - algorytm najbliższego sąsiada",target_url=target_url_krob100)
prepare(nn, 0,"Wynik dla najdłuższej ścieżki - algorytm najbliższego sąsiada",target_url=target_url_krob100)
print()
print(nn)
c = pd.read_csv("krob", index_col=False)
nn = c.loc[c['GREEDY'].isin([11954 ,15392])]
prepare(nn, 0,"Wynik dla najkrótszej ścieżki - algorytm rozbudowy cyklu",target_url=target_url_krob100)
prepare(nn, 1,"Wynik dla najdłuższej ścieżki - algorytm rozbudowy cyklu",target_url=target_url_krob100)
print(nn)
print(nn)
c = pd.read_csv("krob", index_col=False)
nn = c.loc[c['REGRET'].isin([9954,13885])]
prepare(nn, 0,"Wynik dla najkrótszej ścieżki - algorytm oparty na żalu",target_url=target_url_krob100)
prepare(nn, 1,"Wynik dla najdłuższej ścieżki - algorytm oparty na żalu",target_url=target_url_krob100)
print(nn)
